# Good weekends for deep sky viewing at Tammin Observatory

This notebook uses `astropy` and `astroplan` to find good times to visit Tammin Observatory for deep sky observing.

An observing session is considered for each day of the year, starting at astronomical twilight, and ending at 2am (or the end of astronomical twilight, whichever comes first). This is compared to the moon rise and set to find how much of the session is encumbered by moonlight. At the end of the notebook, this list is filtered to Friday and Saturday evenings with less than 90 minutes of moonlight.

This notebook is annotated to explain how the code works.

## Load dependencies

Load required Python modules.

In [1]:
import calendar
from datetime import date, datetime, time, timedelta
from math import ceil
from astroplan import Observer
from astropy.coordinates import EarthLocation
from astropy.time import Time, TimeDelta
import astropy.units as u
import numpy as np
import pandas as pd
from tqdm import tqdm

## Utility functions

These are utility functions used throughout the later code. Each function is documented with a brief description.

In [2]:
def linear_root(x1, y1, x2, y2):
    '''Given points (x1, y1), (x2, y2) such that x1 < x2 and y1 has opposite
    sign to y2, find the point (x, 0) on the straight line between (x1, y1)
    and (x2, y2).'''
    return x1 - y1 * (x2 - x1) / (y2 - y1)


def moon_up_periods(observer, start_time, end_time):
    '''Find all periods for which the moon is risen over a time range.
    Accurate to perhaps 30 minutes at worst, usually better. Returns a list of
    Time() objects with two elements, the rise and set times respectively.'''

    # Ensure moon is down at start and end time
    while observer.moon_altaz(start_time).alt.hour > 0:
        start_time = end_time - TimeDelta(3600, format='sec')
    while observer.moon_altaz(end_time).alt.hour > 0:
        end_time = end_time + TimeDelta(3600, format='sec')

    # Construct a grid of times, calculating moon altitude at each grid time.
    # Search the grid for rises and sets (indicated by changing sign of
    # altitude), then interpolate between the grid times to estimate the
    # rise/set time.
    period = TimeDelta(3600, format='sec')
    n_periods = ceil((end_time - start_time) / period)
    grid = start_time + period * np.linspace(0, n_periods, n_periods + 1)
    moon_alt = observer.moon_altaz(grid).alt.hour
    moon_sign = np.sign(moon_alt)

    rise_times = []
    set_times = []
    for i in range(n_periods):
        if moon_sign[i] == moon_sign[i + 1]:
            continue

        root_time = Time(
            linear_root(
                grid[i].jd, moon_alt[i], grid[i + 1].jd, moon_alt[i + 1]
            ),
            format='jd'
        )
        if moon_sign[i] < moon_sign[i + 1]:
            rise_times.append(root_time)
        else:
            set_times.append(root_time)

    return [Time(x) for x in zip(rise_times, set_times)]


def is_before(x, y):
    '''Given x and y as ordered two element vectors representing two
    intervals on the real line, determine whether x is entirely before y.'''
    return x[1] < y[0]


def is_overlapping(x, y):
    '''Given x and y as ordered two element vectors representing two
    intervals on the real line, determine whether the x and y intervals
    overlap.'''
    return not (y[1] <= x[0] or y[0] >= x[1])


def overlap_timedelta(x, y):
    '''Given x and y as ordered two element Time vectors representing two
    time intervals, return a TimeDelta representing the size of the overlap
    between x and y.'''
    if not is_overlapping(x, y):
        return TimeDelta(0)

    return Time([x[1], y[1]]).min() - Time([x[0], y[0]]).max()


def datetime_combine_astropy_time(observer, d, t):
    '''Convenience function to combine `date` d and `time` t into a Time
    object. The timezone is assumed to be that of the provided observer.'''
    return observer.datetime_to_astropy_time(
        observer.timezone.localize(datetime.combine(d, t))
    )


def seconds_to_zero(xs):
    '''Set number of seconds and milliseconds to zero for each member of
    a sequence of datetime's xs, returning a new list'''
    return [x.replace(second=0, microsecond=0) for x in xs]

## Session parameters

Set the location of the observatory, along with the start and end date to consider.

In [3]:
tammin = Observer(
    location=EarthLocation(lat='-31° 38\'', lon='117° 29\'', height=311 * u.m),
    name='Tammin',
    timezone='Australia/Perth'
)
# Start date is inclusive
start_date = date(2019, 1, 1)
# End date is non inclusive
end_date = date(2020, 1, 1)

## Periods when the moon is above the horizon

Find all times between the start and end date for which the moon is above the horizon. Print the first few periods. These are in Julian date format, so a little hard to read.

In [4]:
moon_periods = moon_up_periods(
    tammin,
    datetime_combine_astropy_time(tammin, start_date, time(0)),
    datetime_combine_astropy_time(tammin, end_date, time(0))
)
print(moon_periods[:5])

[<Time object: scale='utc' format='jd' value=[2458484.23823628 2458484.78691982]>, <Time object: scale='utc' format='jd' value=[2458485.26365085 2458485.82747438]>, <Time object: scale='utc' format='jd' value=[2458486.2908994  2458486.86726734]>, <Time object: scale='utc' format='jd' value=[2458487.31987137 2458487.90582211]>, <Time object: scale='utc' format='jd' value=[2458488.35138275 2458488.94248083]>]


## Evening sessions and moon overlap

For each evening between the start and end date, consider an observation session starting at astronomical twilight, and ending at 2am (or the end of astronomical twilight, whichever comes first). If this session overlaps with any times when the moon is up, note the moon rise and set time and the length of the overlap. 

In [5]:
n_days = (end_date - start_date).days
results_raw = []
remaining_moon_periods = moon_periods.copy()

for offset in tqdm(range(n_days)):
    evening_date = start_date + timedelta(offset)
    midday = datetime_combine_astropy_time(tammin, evening_date, time(12))
    two_am = datetime_combine_astropy_time(
        tammin,
        evening_date + timedelta(1),
        time(2)
    )
    observation_period = Time([
        tammin.twilight_evening_astronomical(midday, 'next'),
        Time([
            two_am,
            tammin.twilight_morning_astronomical(midday, 'next')
        ]).min(),
    ])

    while is_before(remaining_moon_periods[0], observation_period):
        remaining_moon_periods.pop(0)

    overlap1 = overlap_timedelta(
        observation_period,
        remaining_moon_periods[0]
    ).sec
    if len(remaining_moon_periods) > 1:
        overlap2 = overlap_timedelta(
            observation_period,
            remaining_moon_periods[1]
        ).sec
    else:
        overlap2 = 0

    result_raw = {
        'evening_date': evening_date,
        'period_start': observation_period[0],
        'period_end': observation_period[1],
        'period_seconds': (observation_period[1] - observation_period[0]).sec,
        'moon_up_seconds': overlap1 + overlap2,

    }
    if overlap1 > 0:
        result_raw.update({
            'moon_rise1': remaining_moon_periods[0][0],
            'moon_set1': remaining_moon_periods[0][1],
            'moon_rise_illumination1': tammin.moon_illumination(
                remaining_moon_periods[0][0]
            ),
            'moon_up_seconds1': overlap1
        })
    if overlap2 > 0:
        result_raw.update({
            'moon_rise2': remaining_moon_periods[1][0],
            'moon_set2': remaining_moon_periods[1][1],
            'moon_rise_illumination2': tammin.moon_illumination(
                remaining_moon_periods[1][0]
            ),
            'moon_up_seconds2': overlap2
        })
    for column_name in [
        'period_start', 'period_end', 'moon_rise1', 'moon_set1', 'moon_rise2',
        'moon_set2'
    ]:
        if column_name in result_raw:
            result_raw[column_name] = tammin.astropy_time_to_datetime(
                result_raw[column_name]
            )

    results_raw.append(result_raw)

100%|██████████| 365/365 [01:17<00:00,  4.33it/s]


Process the above results into a table in the form of a pandas DataFrame. Show the first three rows as an example.

In [6]:
results = pd.DataFrame.from_dict(results_raw)
results['day_of_week'] = [
    calendar.day_name[evening_date.weekday()]
    for evening_date in results['evening_date']
]
for column in ['period', 'moon_up']:
    results['{}_minutes'.format(column)] = round(
        results['{}_seconds'.format(column)] / 60,
        1
    )
# Omitting moon_rise2 because it never happens here
results = results[[
    'evening_date',
    'day_of_week',
    'period_start',
    'period_end',
    'period_minutes',
    'moon_up_minutes',
    'moon_rise1',
    'moon_set1',
    'moon_rise_illumination1'
]].copy()
for column in ['period_start', 'period_end', 'moon_rise1', 'moon_set1']:
    results[column] = seconds_to_zero(results[column])
    
results.head(3)

,evening_date,day_of_week,period_start,period_end,period_minutes,moon_up_minutes,moon_rise1,moon_set1,moon_rise_illumination1
0,2019-01-01,Tuesday,2019-01-01 20:57:00+08:00,2019-01-02 02:00:00+08:00,302.6,0.0,NaT,NaT,NaN
1,2019-01-02,Wednesday,2019-01-02 20:57:00+08:00,2019-01-03 02:00:00+08:00,302.5,0.0,NaT,NaT,NaN
2,2019-01-03,Thursday,2019-01-03 20:57:00+08:00,2019-01-04 02:00:00+08:00,302.5,0.0,NaT,NaT,NaN


## Good weekends for viewing

Finally, filter the results to Friday and Saturday evenings with less than 90 minutes of moon light. NaT means there is no such time, and NaN means no such number.

In [7]:
filtered_results = results[
    (results.moon_up_minutes <= 90) &
    ((results.day_of_week == 'Friday') | (results.day_of_week == 'Saturday'))
]

filtered_results

,evening_date,day_of_week,period_start,period_end,period_minutes,moon_up_minutes,moon_rise1,moon_set1,moon_rise_illumination1
3,2019-01-04,Friday,2019-01-04 20:57:00+08:00,2019-01-05 02:00:00+08:00,302.5,0.0,NaT,NaT,NaN
4,2019-01-05,Saturday,2019-01-05 20:57:00+08:00,2019-01-06 02:00:00+08:00,302.6,0.0,NaT,NaT,NaN
31,2019-02-01,Friday,2019-02-01 20:42:00+08:00,2019-02-02 02:00:00+08:00,317.6,0.0,NaT,NaT,NaN
32,2019-02-02,Saturday,2019-02-02 20:41:00+08:00,2019-02-03 02:00:00+08:00,318.6,0.0,NaT,NaT,NaN
38,2019-02-08,Friday,2019-02-08 20:35:00+08:00,2019-02-09 02:00:00+08:00,324.9,33.4,2019-02-08 08:27:00+08:00,2019-02-08 21:08:00+08:00,0.087187
39,2019-02-09,Saturday,2019-02-09 20:34:00+08:00,2019-02-10 02:00:00+08:00,326.0,65.2,2019-02-09 09:20:00+08:00,2019-02-09 21:39:00+08:00,0.151004
59,2019-03-01,Friday,2019-03-01 20:08:00+08:00,2019-03-02 02:00:00+08:00,351.0,2.7,2019-03-02 01:57:00+08:00,2019-03-02 16:07:00+08:00,0.201837
60,2019-03-02,Saturday,2019-03-02 20:07:00+08:00,2019-03-03 02:00:00+08:00,352.4,0.0,NaT,NaT,NaN
66,2019-03-08,Friday,2019-03-08 19:59:00+08:00,2019-03-09 02:00:00+08:00,360.5,0.0,NaT,NaT,NaN
67,2019-03-09,Saturday,2019-03-09 19:58:00+08:00,2019-03-10 02:00:00+08:00,361.9,14.4,2019-03-09 08:10:00+08:00,2019-03-09 20:12:00+08:00,0.052263
